In [11]:
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import random

In [12]:
data1='D:\\fintest\\1coal'
data2='D:\\fintest\\2coal'
data3='D:\\fintest\\3coal'
def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([0,0,1])])
        
    shuffle(images)
    return images

data = data_with_label()

random.shuffle(data)
train_data, test_data = train_test_split(data, test_size=0.2)
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1,64,64,1)
tr_img_data = tr_img_data/255
tr_lbl_data = np.array([i[1] for i in train_data])
tst_img_data = np.array([i[0] for i in test_data]).reshape(-1,64,64,1)
tst_img_data = tst_img_data/255
tst_lbl_data = np.array([i[1] for i in test_data])


100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [00:30<00:00, 14.76it/s]


In [15]:
# Loads the weights
def create_model():
  model=Sequential()
  model.add(InputLayer(input_shape=[64,64,1]))

  model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=32,kernel_size=5,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=8,padding='same'))

  model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          

  model.add(Flatten())


  model.add(Dense(2048,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(1024,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(3,activation='softmax'))
  optimizer=Adam(lr=1e-3)
          
  model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
  return model

checkpoint_path = "cp.ckpt"
model=create_model()

model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(tst_img_data,  tst_lbl_data, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 96.94%


In [16]:
for img in tst_img_data:

    img =img.reshape(-1,64,64,1)
    model_out=model.predict([img])
    print(model_out)
    if np.argmax(model_out)==0:
        str_label='1'
        
    elif np.argmax(model_out)==1:
        str_label='2'
    else:
        str_label='3'
    print(str_label)
    

[[1.1980481e-13 3.6093891e-08 1.0000000e+00]]
3
[[0.00322967 0.9938344  0.00293593]]
2
[[4.6526772e-04 9.9564809e-01 3.8866159e-03]]
2
[[1.8617492e-10 3.1684863e-06 9.9999678e-01]]
3
[[5.2945473e-04 9.9539775e-01 4.0728347e-03]]
2
[[9.9986196e-01 1.3803068e-04 2.6540506e-11]]
1
[[0.00106064 0.9915901  0.00734932]]
2
[[9.9999976e-01 1.7928338e-07 1.9651007e-17]]
1
[[9.7808892e-01 2.1910178e-02 9.3823110e-07]]
1
[[9.9917799e-01 8.2195387e-04 1.6251442e-09]]
1
[[8.5298182e-04 9.7865021e-01 2.0496868e-02]]
2
[[4.5407414e-06 4.3543568e-03 9.9564111e-01]]
3
[[9.0886565e-04 9.9799621e-01 1.0949789e-03]]
2
[[0.00663546 0.8278085  0.16555601]]
2
[[2.0143985e-16 4.7432808e-10 1.0000000e+00]]
3
[[1.9402555e-21 2.0870811e-13 1.0000000e+00]]
3
[[5.5246377e-11 1.3807771e-06 9.9999857e-01]]
3
[[5.2479655e-04 9.9887949e-01 5.9572817e-04]]
2
[[9.9998403e-01 1.5976626e-05 2.2221932e-13]]
1
[[9.9974972e-01 2.5024303e-04 1.1018965e-10]]
1
[[5.4087383e-09 3.0790132e-05 9.9996924e-01]]
3
[[9.9957079e-01 4.2

[[1.1993155e-03 9.9784827e-01 9.5247576e-04]]
2
[[9.9806052e-01 1.9394729e-03 7.0263440e-09]]
1
[[9.9998868e-01 1.1323582e-05 1.2228387e-13]]
1
[[9.9987674e-01 1.2326357e-04 1.8809189e-11]]
1
[[0.00176412 0.9957748  0.00246118]]
2
[[1.9131943e-13 2.3329852e-08 1.0000000e+00]]
3
[[5.3738050e-14 1.3775841e-08 1.0000000e+00]]
3
[[0.00399767 0.99398637 0.00201596]]
2
[[9.9999523e-01 4.7757944e-06 1.7681376e-14]]
1
[[1.5993738e-15 2.1676108e-09 1.0000000e+00]]
3
[[9.2004692e-13 1.3231687e-07 9.9999988e-01]]
3
[[9.9994385e-01 5.6179717e-05 4.8380609e-12]]
1
[[2.3791124e-04 9.9809521e-01 1.6668586e-03]]
2
[[9.3335849e-01 6.6639096e-02 2.4199801e-06]]
1
[[9.99996185e-01 3.76106118e-06 1.17242845e-14]]
1
[[1.5955835e-07 3.6792009e-04 9.9963188e-01]]
3
[[7.9354522e-04 9.9764520e-01 1.5613431e-03]]
2
[[7.2587746e-17 2.6169764e-10 1.0000000e+00]]
3
[[5.588066e-04 9.986694e-01 7.718115e-04]]
2
[[2.4658658e-10 5.5241394e-06 9.9999452e-01]]
3
[[1.4167086e-03 9.9785620e-01 7.2709931e-04]]
2
[[9.931498